In [1]:
import numpy as np
import pandas as pd

In [2]:
movies=pd.read_csv('movies.csv',usecols=['movieId','title'])
ratings=pd.read_csv('ratings.csv',usecols=['userId','movieId','rating'])
data=pd.merge(ratings,movies,on='movieId')
movie_ratings=(data.groupby(by=['title'])['rating'].count().reset_index().rename(columns={'rating':'total_ratings'}))
data=pd.merge(data,movie_ratings,on='title')
data['total_ratings'].describe()
data=data[data['total_ratings'] > 50]
table=data.pivot_table(index='title',columns='userId',values='rating').fillna(0)


In [3]:
table.head(1)

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0


In [4]:
from scipy.sparse import csr_matrix
points = csr_matrix(table.values)

In [5]:
print(points)

  (0, 11)	5.0
  (0, 18)	3.0
  (0, 66)	4.5
  (0, 90)	5.0
  (0, 102)	5.0
  (0, 109)	4.0
  (0, 130)	4.0
  (0, 151)	1.0
  (0, 156)	3.5
  (0, 174)	4.5
  (0, 179)	4.0
  (0, 195)	1.0
  (0, 197)	5.0
  (0, 216)	2.0
  (0, 233)	5.0
  (0, 246)	3.5
  (0, 257)	4.5
  (0, 271)	3.0
  (0, 272)	4.0
  (0, 277)	4.0
  (0, 283)	3.0
  (0, 295)	2.5
  (0, 304)	0.5
  (0, 313)	3.0
  (0, 322)	3.0
  :	:
  (436, 378)	2.0
  (436, 384)	3.0
  (436, 410)	0.5
  (436, 411)	4.0
  (436, 416)	4.0
  (436, 417)	4.0
  (436, 421)	3.0
  (436, 423)	2.0
  (436, 435)	2.5
  (436, 444)	3.0
  (436, 476)	4.5
  (436, 479)	5.0
  (436, 480)	4.5
  (436, 485)	1.5
  (436, 491)	5.0
  (436, 521)	4.0
  (436, 530)	4.0
  (436, 538)	1.5
  (436, 556)	4.0
  (436, 557)	3.0
  (436, 569)	5.0
  (436, 585)	2.5
  (436, 595)	4.5
  (436, 603)	3.0
  (436, 605)	4.0


In [6]:
from sklearn.neighbors import NearestNeighbors
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(points)

NearestNeighbors(algorithm='brute', metric='cosine')

In [7]:
distances, indices = model_knn.kneighbors(table.iloc[200].values.reshape(1, -1), n_neighbors = 10)
print(distances,indices)

[[0.         0.3138237  0.37973146 0.44040063 0.44724591 0.46693049
  0.47444272 0.47514396 0.4824556  0.48299232]] [[200 342 382 435  81 344  44  89  24 351]]


In [8]:
table.index.values[0]

'10 Things I Hate About You (1999)'

In [9]:
for i in range(0, len(distances[0])):
    if i == 0:
        print('Recommendations for {0}:\n'.format(table.index[1]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, table.index[indices[0][i]], distances[0][i]))


Recommendations for 12 Angry Men (1957):

1: Shaun of the Dead (2004), with distance of 0.31382370004269133:
2: Superbad (2007), with distance of 0.37973145855538504:
3: Zombieland (2009), with distance of 0.44040062622041143:
4: Casino Royale (2006), with distance of 0.4472459073266031:
5: Sherlock Holmes (2009), with distance of 0.46693049182809276:
6: Batman Begins (2005), with distance of 0.47444271734516363:
7: Children of Men (2006), with distance of 0.4751439594598079:
8: Anchorman: The Legend of Ron Burgundy (2004), with distance of 0.48245559894494305:
9: Sin City (2005), with distance of 0.48299232006606285:


In [10]:
query = "Sin City (2005)" 

if query in table.index.values:
    index_of_title = table.index.get_loc(query)
    print("Index of", query, ":", index_of_title)
else:
    print("Title not found in the DataFrame.")


Index of Sin City (2005) : 351
